In [4]:
# !pip install folium
# !pip install geocoder

In [7]:
# Import libs
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import re as re
from pandas.io.json import json_normalize
import json # library to handle JSON files
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import geocoder
import geopandas

ModuleNotFoundError: No module named 'folium'

# Prepare neighborhood list: Manhattan

In [88]:
url = 'https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods'
results = requests.get(url)
soup = BeautifulSoup(results.content)

In [89]:
MA_info = soup.find_all('table', attrs = {'class':'wikitable sortable'})

In [90]:
MA_list = []
for area in MA_info:
    list_data = area.tbody.find_all('tr')
    for row in list_data[1:]:
        MA_list.append(row.find('td').text.replace('\n', '').strip())
    for index, neighborhood in enumerate(MA_list):
        try:
            MA_list[index] = re.split('[(,\[†]', neighborhood)[0].strip()
        except:
            pass
    for index, neighborhood in enumerate(MA_list):
        try:
            MA_list[index] = neighborhood.split(' aka ')[0].strip()
        except:
            pass
MA_list[-3] = MA_list[-3].replace('Historical District','').strip()
MA_list[62] = MA_list[62].replace('and Loisaida', '').strip()
MA_list

['Upper Manhattan',
 'Marble Hill',
 'Inwood',
 'Fort George',
 'Washington Heights',
 'Hudson Heights',
 'West Harlem',
 'Hamilton Heights',
 'Manhattanville',
 'Morningside Heights',
 'Central Harlem',
 'Harlem',
 'St. Nicholas Historic District',
 'Astor Row',
 'Sugar Hill',
 'Marcus Garvey Park',
 'Le Petit Senegal',
 'East Harlem',
 'Upper East Side',
 'Lenox Hill',
 'Carnegie Hill',
 'Yorkville',
 'Upper West Side',
 'Manhattan Valley',
 'Lincoln Square',
 'Columbus Circle',
 'Sutton Place',
 'Rockefeller Center',
 'Diamond District',
 'Theater District',
 'Turtle Bay',
 'Midtown East',
 'Midtown',
 'Tudor City',
 'Little Brazil',
 'Times Square',
 'Hudson Yards',
 'Midtown West',
 "Hell's Kitchen",
 'Garment District',
 'Herald Square',
 'Koreatown',
 'Murray Hill',
 'Tenderloin',
 'Madison Square',
 'Flower District',
 'Brookdale',
 'Hudson Yards',
 'Kips Bay',
 'Rose Hill',
 'NoMad',
 'Peter Cooper Village',
 'Chelsea',
 'Flatiron District',
 'Gramercy Park',
 'Stuyvesant Squa

# Prepare neighborhood list: DC

In [81]:
url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Administrative_Other_Boundaries_WebMercator/MapServer/35/query?where=1%3D1&outFields=NAME&outSR=4326&f=json'
results = requests.get(url).json()
DC_list_full = json_normalize(results['features'])
DC_list_full.columns = ['Name', 'Latitude', 'Longitude']

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


In [82]:
DC_list_full['Name'] = DC_list_full['Name'].str.replace('-',' ', regex=False)

def name_correct(name, delimit='/'):
    return name.split(delimit)[0]

DC_list_full['Name'] = DC_list_full['Name'].apply(name_correct)
DC_list = DC_list_full['Name'].tolist()
DC_list

['Fort Stanton',
 'Congress Heights',
 'Washington Highlands',
 'Bellevue',
 'Knox Hill',
 'Shipley',
 'Douglass',
 'Woodland',
 'Garfield Heights',
 'Near Southeast',
 'Capitol Hill',
 'Dupont Park',
 'Twining',
 'Randle Highlands',
 'Fairlawn',
 'Penn Branch',
 'Barry Farm',
 'Historic Anacostia',
 'Columbia Heights',
 'Logan Circle',
 'Cardozo',
 'Van Ness',
 'Forest Hills',
 'Georgetown Reservoir',
 'Foxhall Village',
 'Fort Totten',
 'Pleasant Hill',
 'Kenilworth',
 'Eastland Gardens',
 'Deanwood',
 'Fort Dupont',
 'Greenway',
 'Woodland Normanstone',
 'Mass. Ave. Heights',
 'Naylor Gardens',
 'Pleasant Plains',
 'Hillsdale',
 'Benning Ridge',
 'Penn Quarter',
 'Chinatown',
 'Stronghold',
 'South Central',
 'Langston',
 'Downtown East',
 'North Portal Estates',
 'Colonial Village',
 'Shepherd Park',
 'Takoma',
 'Lamond Riggs',
 'Petworth',
 'Brightwood Park',
 'Manor Park',
 'Brightwood',
 'Hawthorne',
 'Barnaby Woods',
 'Queens Chapel',
 'Michigan Park',
 'North Michigan Park',
 

# Prepare neighborhood list: Shanghai

In [69]:
url = 'https://en.wikipedia.org/wiki/Jing%27an_District'
results = requests.get(url)
soup = BeautifulSoup(results.content)

In [70]:
SH1_info = soup.find('table', attrs = {'class':'wikitable'}).tbody.find_all('tr')

In [71]:
SH1_list = []
for row in SH1_info[1:]:
    SH1_list.append(row.find('td').text.replace('\n','').strip())
for index, neighborhood in enumerate(SH1_list):
    try:
        SH1_list[index] = re.split('[\[]', neighborhood)[0].strip()
    except:
        pass
SH1_list

['Beizhan Subdistrict',
 'Baoshan Road Subdistrict',
 'Caojiadu Subdistrict',
 'Daning Road Subdistrict',
 'Gonghexin Road Subdistrict',
 'Jiangning Road Subdistrict',
 "Jing'ansi Subdistrict",
 'Linfen Road Subdistrict',
 'Pengpu Xincun Subdistrict',
 'Shimen Second Road Subdistrict',
 'Nanjing West Road Subdistrict',
 'Tianmu West Road Subdistrict',
 'Zhijiang West Road Subdistrict',
 'Pengpu town']

In [72]:
url = 'https://en.wikipedia.org/wiki/Hongkou_District'
results = requests.get(url)
soup = BeautifulSoup(results.content)

In [73]:
SH2_info = soup.find('table', attrs = {'class':'wikitable'}).tbody.find_all('tr')

In [74]:
SH2_list = []
for row in SH2_info[1:]:
    SH2_list.append(row.find('td').text.replace('\n','').strip())
for index, neighborhood in enumerate(SH2_list):
    try:
        SH2_list[index] = re.split('[\[]', neighborhood)[0].strip()
    except:
        pass
SH2_list

['Ouyang Road Subdistrict',
 'Quyang Road Subdistrict',
 'Guangzhong Road Subdistrict',
 'Jiaxing Road Subdistrict',
 'Liangcheng Xincun Subdistrict',
 'Sichuan North Road Subdistrict',
 'Tilanqiao Subdistrict',
 'Jiangwanzhen Subdistrict']

In [75]:
url = 'https://en.wikipedia.org/wiki/Huangpu_District,_Shanghai'
results = requests.get(url)
soup = BeautifulSoup(results.content)

In [76]:
SH3_info = soup.find('table', attrs = {'class':'wikitable'}).tbody.find_all('tr')

In [77]:
SH3_list = []
for row in SH3_info[1:]:
    SH3_list.append(row.find('td').text.replace('\n','').strip())
for index, neighborhood in enumerate(SH3_list):
    try:
        SH3_list[index] = re.split('[\[]', neighborhood)[0].strip()
    except:
        pass
SH3_list

['Bansongyuan Road Subdistrict',
 'Huaihai Central Road Subdistrict',
 'Dapuqiao Subdistrict',
 'Nanjing East Road Subdistrict',
 'Laoximen Subdistrict',
 'Ruijin Second Road Subdistrict',
 'Wuliqiao Subdistrict',
 'Xiaodongmen Subdistrict',
 'Yuyuan Subdistrict']

In [173]:
SH_list = SH1_list + SH2_list + SH3_list
len(SH_list)

31

# Prepare neighborhood list: Beijing

In [9]:
url = 'https://en.wikipedia.org/wiki/Xicheng_District'
results = requests.get(url)
soup = BeautifulSoup(results.content)

In [10]:
BJ1_info = soup.find('table', attrs = {'class':'wikitable'}).tbody.find_all('tr')

In [11]:
BJ1_list = []
for row in BJ1_info[1:]:
    BJ1_list.append(row.find('td').text.replace('\n','').strip())
for index, neighborhood in enumerate(BJ1_list):
    try:
        BJ1_list[index] = re.split('[\[]', neighborhood)[0].strip()
    except:
        pass
BJ1_list

['Jinrong Street Subdistrict',
 "West Chang'an Avenue Subdistrict",
 'Xinjiekou Subdistrict',
 'Yuetan Subdistrict',
 'Zhanlan Road Subdistrict',
 'Desheng Subdistrict',
 'Shichahai Subdistrict',
 'Dashilan Subdistrict',
 'Tianqiao Subdistrict',
 'Chunshu Subdistrict',
 'Taoranting Subdistrict',
 "Guang'anmennei Subdistrict",
 'Niujie Subdistrict',
 'Baizhifang Subdistrict',
 "Guang'anmenwai Subdistrict"]

In [12]:
url = 'https://en.wikipedia.org/wiki/Dongcheng_District,_Beijing'
results = requests.get(url)
soup = BeautifulSoup(results.content)

In [13]:
BJ2_info = soup.find('table', attrs = {'class':'wikitable'}).tbody.find_all('tr')

In [14]:
BJ2_list = []
for row in BJ2_info[1:]:
    BJ2_list.append(row.find('td').text.replace('\n','').strip())
for index, neighborhood in enumerate(BJ2_list):
    try:
        BJ2_list[index] = re.split('[\[]', neighborhood)[0].strip()
    except:
        pass
BJ2_list

['Jingshan Subdistrict',
 'Donghuamen Subdistrict',
 'Jiaodoukou Subdistrict',
 'Andingmen Subdistrict',
 'Beixinqiao Subdistrict',
 'Dongsi Subdistrict',
 'Chaoyangmen Subdistrict',
 'Jianguomen Subdistrict',
 'Dongzhimen Subdistrict',
 'Hepingli Subdistrict',
 'Qianmen Subdistrict',
 'Chongwenmenwai Subdistrict',
 'Donghuashi Subdistrict',
 'Longtan Subdistrict',
 'Tiyuguan Road Subdistrict',
 'Tiantan Subdistrict',
 'Yongdingmenwai Subdistrict']

In [15]:
BJ_list = BJ1_list + BJ2_list

In [16]:
BJ_list

['Jinrong Street Subdistrict',
 "West Chang'an Avenue Subdistrict",
 'Xinjiekou Subdistrict',
 'Yuetan Subdistrict',
 'Zhanlan Road Subdistrict',
 'Desheng Subdistrict',
 'Shichahai Subdistrict',
 'Dashilan Subdistrict',
 'Tianqiao Subdistrict',
 'Chunshu Subdistrict',
 'Taoranting Subdistrict',
 "Guang'anmennei Subdistrict",
 'Niujie Subdistrict',
 'Baizhifang Subdistrict',
 "Guang'anmenwai Subdistrict",
 'Jingshan Subdistrict',
 'Donghuamen Subdistrict',
 'Jiaodoukou Subdistrict',
 'Andingmen Subdistrict',
 'Beixinqiao Subdistrict',
 'Dongsi Subdistrict',
 'Chaoyangmen Subdistrict',
 'Jianguomen Subdistrict',
 'Dongzhimen Subdistrict',
 'Hepingli Subdistrict',
 'Qianmen Subdistrict',
 'Chongwenmenwai Subdistrict',
 'Donghuashi Subdistrict',
 'Longtan Subdistrict',
 'Tiyuguan Road Subdistrict',
 'Tiantan Subdistrict',
 'Yongdingmenwai Subdistrict']

# Request neighborhood position

## Mahattan

In [131]:
lat = []
lng = []
for neighborhood in MA_list:
    address = '{}, New York, NY'.format(neighborhood)
    g = geocoder.arcgis(address)
    lat.append(g.lat)
    lng.append(g.lng)

In [132]:
MA_table = pd.DataFrame(MA_list)
MA_table.columns = ['Neighborhoods']
MA_table

,Neighborhoods
0,Upper Manhattan
1,Marble Hill
2,Inwood
3,Fort George
4,Washington Heights
...,...
79,Civic Center
80,Radio Row
81,South Street Seaport
82,Battery Park City


In [133]:
MA_table['Latitude'] = lat
MA_table['Longitude'] = lng
MA_table

,Neighborhoods,Latitude,Longitude
0,Upper Manhattan,40.808810,-73.947770
1,Marble Hill,40.876410,-73.911000
2,Inwood,40.865950,-73.926070
3,Fort George,40.859187,-73.926459
4,Washington Heights,40.849830,-73.936030
...,...,...,...
79,Civic Center,40.713370,-74.003800
80,Radio Row,40.843567,-73.818163
81,South Street Seaport,40.705710,-74.001840
82,Battery Park City,40.711310,-74.015900


In [137]:
for row in range(MA_table.shape[0]):
    if (MA_table.loc[row, 'Latitude'] <= 40.65526141744085) or (MA_table.loc[row, 'Longitude'] >= -73.81816282365787):
        MA_table.drop(row, inplace=True)
MA_table.reset_index(drop=True, inplace=True)
MA_table

,Neighborhoods,Latitude,Longitude
0,Upper Manhattan,40.808810,-73.947770
1,Marble Hill,40.876410,-73.911000
2,Inwood,40.865950,-73.926070
3,Fort George,40.859187,-73.926459
4,Washington Heights,40.849830,-73.936030
...,...,...,...
73,Tribeca,40.718540,-74.008880
74,Civic Center,40.713370,-74.003800
75,South Street Seaport,40.705710,-74.001840
76,Battery Park City,40.711310,-74.015900


## Map of Mahattan

In [171]:
address = 'New York, NY'

g = geocoder.arcgis(address)
print(g.lat, g.lng)

40.71455000000003 -74.00713999999994


In [172]:
map_newyork = folium.Map(location=[g.lat, g.lng], zoom_start=12)

for neighborhood, lat, lng in zip(MA_table['Neighborhoods'], MA_table['Latitude'], MA_table['Longitude']):
    label = '{},{}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_newyork)
map_newyork

## Request venues from Foursquare

### Define a required function

In [143]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    neighborhood_venues_info = []
    venues_info = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        neighborhood_venues_info.append(results)
        venues_info.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        print(len(venues_info), ':', name)

    nearby_venues = pd.DataFrame([item for venue_info in venues_info for item in venue_info])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues, venues_info, neighborhood_venues_info)

### Define Foursquare credential and version

In [139]:
CLIENT_ID = 'OQWZRAUBBN5XX5CTG4QJNUTGB1PLERCI32GLZWVRG4JJTOMN' # your Foursquare ID
CLIENT_SECRET = '5J0EYVSGK2QSQXGA4TN5YPIIUY1NU0HDIRQNHRWWB4BXUUAR' # your Foursquare Secret
ACCESS_TOKEN = 'VH53ZCZMIQSN033ZU2VDUZD1CRA5GJQE1SKGVRWNNF1FJINZ' # your FourSquare Access Token
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OQWZRAUBBN5XX5CTG4QJNUTGB1PLERCI32GLZWVRG4JJTOMN
CLIENT_SECRET:5J0EYVSGK2QSQXGA4TN5YPIIUY1NU0HDIRQNHRWWB4BXUUAR


In [144]:
names = MA_table['Neighborhoods'].tolist()
latitudes = MA_table['Latitude'].tolist()
longitudes = MA_table['Longitude'].tolist()
MA_venues, MA_venues_info, MA_neighborhood_venues_info = getNearbyVenues(names, latitudes=latitudes, longitudes=longitudes)

1 : Upper Manhattan
2 : Marble Hill
3 : Inwood
4 : Fort George
5 : Washington Heights
6 : Hudson Heights
7 : West Harlem
8 : Hamilton Heights
9 : Manhattanville
10 : Morningside Heights
11 : Central Harlem
12 : Harlem
13 : St. Nicholas Historic District
14 : Astor Row
15 : Sugar Hill
16 : Marcus Garvey Park
17 : Le Petit Senegal
18 : East Harlem
19 : Upper East Side
20 : Lenox Hill
21 : Carnegie Hill
22 : Yorkville
23 : Upper West Side
24 : Manhattan Valley
25 : Lincoln Square
26 : Columbus Circle
27 : Sutton Place
28 : Rockefeller Center
29 : Diamond District
30 : Theater District
31 : Turtle Bay
32 : Midtown East
33 : Midtown
34 : Tudor City
35 : Little Brazil
36 : Times Square
37 : Hudson Yards
38 : Midtown West
39 : Hell's Kitchen
40 : Garment District
41 : Herald Square
42 : Koreatown
43 : Tenderloin
44 : Madison Square
45 : Hudson Yards
46 : Kips Bay
47 : Rose Hill
48 : NoMad
49 : Peter Cooper Village
50 : Flatiron District
51 : Gramercy Park
52 : Stuyvesant Square
53 : Union Squ

___
## DC

In [146]:
lat = []
lng = []
for neighborhood in DC_list:
    address = '{}, Washington, DC'.format(neighborhood)
    g = geocoder.arcgis(address)
    lat.append(g.lat)
    lng.append(g.lng)

In [147]:
DC_table = pd.DataFrame(DC_list)
DC_table.columns = ['Neighborhoods']
DC_table

,Neighborhoods
0,Fort Stanton
1,Congress Heights
2,Washington Highlands
3,Bellevue
4,Knox Hill
...,...
127,Fort Davis Park
128,Fairfax Village
129,Hillcrest
130,Crestwood


In [148]:
DC_table['Latitude'] = lat
DC_table['Longitude'] = lng
DC_table

,Neighborhoods,Latitude,Longitude
0,Fort Stanton,38.85760,-76.97830
1,Congress Heights,38.84290,-77.00057
2,Washington Highlands,38.83198,-76.99700
3,Bellevue,38.82654,-77.00870
4,Knox Hill,38.85302,-76.97246
...,...,...,...
127,Fort Davis Park,38.86794,-76.94493
128,Fairfax Village,38.86372,-76.95220
129,Hillcrest,38.85941,-76.95566
130,Crestwood,38.94187,-77.03989


In [137]:
# for row in range(MA_table.shape[0]):
#     if (MA_table.loc[row, 'Latitude'] <= 40.65526141744085) or (MA_table.loc[row, 'Longitude'] >= -73.81816282365787):
#         MA_table.drop(row, inplace=True)
# MA_table.reset_index(drop=True, inplace=True)
# MA_table

,Neighborhoods,Latitude,Longitude
0,Upper Manhattan,40.808810,-73.947770
1,Marble Hill,40.876410,-73.911000
2,Inwood,40.865950,-73.926070
3,Fort George,40.859187,-73.926459
4,Washington Heights,40.849830,-73.936030
...,...,...,...
73,Tribeca,40.718540,-74.008880
74,Civic Center,40.713370,-74.003800
75,South Street Seaport,40.705710,-74.001840
76,Battery Park City,40.711310,-74.015900


## Map of DC

In [168]:
address = 'Washington, DC'
g = geocoder.arcgis(address)
print(g.lat, g.lng)

38.890370000000075 -77.03195999999997


In [169]:
map_DC = folium.Map(location=[g.lat, g.lng], zoom_start=12)

for neighborhood, lat, lng in zip(DC_table['Neighborhoods'], DC_table['Latitude'], DC_table['Longitude']):
    label = '{},{}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_DC)
map_DC

### Define Foursquare credential and version

In [154]:
names = DC_table['Neighborhoods'].tolist()
latitudes = DC_table['Latitude'].tolist()
longitudes = DC_table['Longitude'].tolist()
DC_venues, DC_venues_info, DC_neighborhood_venues_info = getNearbyVenues(names, latitudes=latitudes, longitudes=longitudes)

1 : Fort Stanton
2 : Congress Heights
3 : Washington Highlands
4 : Bellevue
5 : Knox Hill
6 : Shipley
7 : Douglass
8 : Woodland
9 : Garfield Heights
10 : Near Southeast
11 : Capitol Hill
12 : Dupont Park
13 : Twining
14 : Randle Highlands
15 : Fairlawn
16 : Penn Branch
17 : Barry Farm
18 : Historic Anacostia
19 : Columbia Heights
20 : Logan Circle
21 : Cardozo
22 : Van Ness
23 : Forest Hills
24 : Georgetown Reservoir
25 : Foxhall Village
26 : Fort Totten
27 : Pleasant Hill
28 : Kenilworth
29 : Eastland Gardens
30 : Deanwood
31 : Fort Dupont
32 : Greenway
33 : Woodland Normanstone
34 : Mass. Ave. Heights
35 : Naylor Gardens
36 : Pleasant Plains
37 : Hillsdale
38 : Benning Ridge
39 : Penn Quarter
40 : Chinatown
41 : Stronghold
42 : South Central
43 : Langston
44 : Downtown East
45 : North Portal Estates
46 : Colonial Village
47 : Shepherd Park
48 : Takoma
49 : Lamond Riggs
50 : Petworth
51 : Brightwood Park
52 : Manor Park
53 : Brightwood
54 : Hawthorne
55 : Barnaby Woods
56 : Queens Cha

___
## Beijing

In [17]:
lat = []
lng = []
for neighborhood in BJ_list:
    address = '{}, Beijing, China'.format(neighborhood)
    g = geocoder.arcgis(address)
    lat.append(g.lat)
    lng.append(g.lng)

In [23]:
g.geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'address': 'Beijing',
    'bbox': [116.12223000000003,
     39.63250000000003,
     116.67223000000004,
     40.182500000000026],
    'confidence': 1,
    'lat': 39.90750000000003,
    'lng': 116.39723000000004,
    'ok': True,
    'quality': 'Locality',
    'raw': {'name': 'Beijing',
     'extent': {'xmin': 116.12223000000003,
      'ymin': 39.63250000000003,
      'xmax': 116.67223000000004,
      'ymax': 40.182500000000026},
     'feature': {'geometry': {'x': 116.39723000000004, 'y': 39.90750000000003},
      'attributes': {'Score': 100, 'Addr_Type': 'Locality'}}},
    'score': 100,
    'status': 'OK'},
   'bbox': [116.12223000000003,
    39.63250000000003,
    116.67223000000004,
    40.182500000000026],
   'geometry': {'type': 'Point',
    'coordinates': [116.39723000000004, 39.90750000000003]}}]}

In [18]:
BJ_table = pd.DataFrame(BJ_list)
BJ_table.columns = ['Neighborhoods']
BJ_table

,Neighborhoods
0,Jinrong Street Subdistrict
1,West Chang'an Avenue Subdistrict
2,Xinjiekou Subdistrict
3,Yuetan Subdistrict
4,Zhanlan Road Subdistrict
5,Desheng Subdistrict
6,Shichahai Subdistrict
7,Dashilan Subdistrict
8,Tianqiao Subdistrict
9,Chunshu Subdistrict


In [19]:
BJ_table['Latitude'] = lat
BJ_table['Longitude'] = lng
BJ_table

,Neighborhoods,Latitude,Longitude
0,Jinrong Street Subdistrict,39.907500,116.397230
1,West Chang'an Avenue Subdistrict,39.907500,116.397230
2,Xinjiekou Subdistrict,39.939790,116.366910
3,Yuetan Subdistrict,39.908470,116.332000
4,Zhanlan Road Subdistrict,39.907500,116.397230
5,Desheng Subdistrict,39.907500,116.397230
6,Shichahai Subdistrict,39.938110,116.390710
7,Dashilan Subdistrict,39.907500,116.397230
8,Tianqiao Subdistrict,33.564500,112.731100
9,Chunshu Subdistrict,39.894460,116.368910


In [137]:
# for row in range(MA_table.shape[0]):
#     if (MA_table.loc[row, 'Latitude'] <= 40.65526141744085) or (MA_table.loc[row, 'Longitude'] >= -73.81816282365787):
#         MA_table.drop(row, inplace=True)
# MA_table.reset_index(drop=True, inplace=True)
# MA_table

,Neighborhoods,Latitude,Longitude
0,Upper Manhattan,40.808810,-73.947770
1,Marble Hill,40.876410,-73.911000
2,Inwood,40.865950,-73.926070
3,Fort George,40.859187,-73.926459
4,Washington Heights,40.849830,-73.936030
...,...,...,...
73,Tribeca,40.718540,-74.008880
74,Civic Center,40.713370,-74.003800
75,South Street Seaport,40.705710,-74.001840
76,Battery Park City,40.711310,-74.015900


## Map of Beijing

In [20]:
address = 'Beijing, China'
g = geocoder.arcgis(address)
print(g.lat, g.lng)

39.90750000000003 116.39723000000004


In [21]:
map_BJ = folium.Map(location=[g.lat, g.lng], zoom_start=12)

for neighborhood, lat, lng in zip(BJ_table['Neighborhoods'], BJ_table['Latitude'], BJ_table['Longitude']):
#     label = '{},{}'.format(lat, lng)
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_BJ)
map_BJ

### Define Foursquare credential and version

In [154]:
names = BJ_table['Neighborhoods'].tolist()
latitudes = BJ_table['Latitude'].tolist()
longitudes = BJ_table['Longitude'].tolist()
BJ_venues, BJ_venues_info, BJ_neighborhood_venues_info = getNearbyVenues(names, latitudes=latitudes, longitudes=longitudes)

1 : Fort Stanton
2 : Congress Heights
3 : Washington Highlands
4 : Bellevue
5 : Knox Hill
6 : Shipley
7 : Douglass
8 : Woodland
9 : Garfield Heights
10 : Near Southeast
11 : Capitol Hill
12 : Dupont Park
13 : Twining
14 : Randle Highlands
15 : Fairlawn
16 : Penn Branch
17 : Barry Farm
18 : Historic Anacostia
19 : Columbia Heights
20 : Logan Circle
21 : Cardozo
22 : Van Ness
23 : Forest Hills
24 : Georgetown Reservoir
25 : Foxhall Village
26 : Fort Totten
27 : Pleasant Hill
28 : Kenilworth
29 : Eastland Gardens
30 : Deanwood
31 : Fort Dupont
32 : Greenway
33 : Woodland Normanstone
34 : Mass. Ave. Heights
35 : Naylor Gardens
36 : Pleasant Plains
37 : Hillsdale
38 : Benning Ridge
39 : Penn Quarter
40 : Chinatown
41 : Stronghold
42 : South Central
43 : Langston
44 : Downtown East
45 : North Portal Estates
46 : Colonial Village
47 : Shepherd Park
48 : Takoma
49 : Lamond Riggs
50 : Petworth
51 : Brightwood Park
52 : Manor Park
53 : Brightwood
54 : Hawthorne
55 : Barnaby Woods
56 : Queens Cha

In [18]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-463aa675-2b0c-4076-83c6-caf2586fe4af',
    'IBM_API_KEY_ID': 'EHx0iauXQDqHaspTvW4gAnjncGznQb0-nWtaxMbxdeu7',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'pythonmlfinalassign-donotdelete-pr-t1d0rs96kjtnmp',
    'FILE': 'china-provinces.json'
}


In [22]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ef325d89bbab4c4d98e512cdf0706b1e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='EHx0iauXQDqHaspTvW4gAnjncGznQb0-nWtaxMbxdeu7',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ef325d89bbab4c4d98e512cdf0706b1e.get_object(Bucket='pythonmlfinalassign-donotdelete-pr-t1d0rs96kjtnmp',Key='china-provinces.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Please read the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' to learn more about the possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

df_data_1 = pandas.io.json.json_normalize(body)
# df_data_1.head()


NameError: name 'pandas' is not defined

In [21]:
df_data_1 = pd.read_json(body, orient='values')


ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.